In [1]:
import textrank
import pysolr
import uuid
import os
import time
import nltk
import json
import nltk.data

import sys
sys.stdout.encoding
'UTF-8'

'UTF-8'

In [2]:
LANGUAGE= 'spanish'

SOLR_URL = "http://argon-solr:8975/solr/opendata"
solr = pysolr.Solr(SOLR_URL, timeout=10)

In [3]:
# retrieves a document set by category (from Solr).
def retrieve_documents_solr(category, k = 100000):
    document_set = []    
    query = solr.search(q= 'categorias:' + '\"' + category + '\"', fl="texto", rows = k)
    for q in query:
        document_set.append(q['texto'])
    return document_set

#category = '[2-GeneralCategorization/Health]http://opendata.aragon.es/def/ei2a/categorization#Health'
#print retrieve_documents_solr(category, k = 1)

In [4]:
# concatenates all retrieved documents into a single document, called corpus.
def concat_documents(document_set):    
    corpus = ''
    for document in document_set:
        corpus += document + ' '       
    return corpus

In [5]:
def text_rank(corpus, k):
    #start_time = time.time()
    keyphrases = textrank.extractKeyphrases(text = corpus, k = k, lang = LANGUAGE, download_dir='/home/jovyan/work/data/nltk/')   #/home/jovyan/work/data/nltk/
    #print("--- %s TextRank seconds ---" % (time.time() - start_time))
    return keyphrases

In [6]:
def get_keyphrases(category, number_documents, number_phrases):  
    document_set = retrieve_documents_solr(category, number_documents)
    
    #TextRank:
    corpus = concat_documents(document_set)
    keyphrases = text_rank(corpus = corpus, k = number_phrases)

    return keyphrases

# category = "[2-GeneralCategorization/Health]http://opendata.aragon.es/def/ei2a/categorization#Health"
# numberdocuments = 2
# numberphrases = 20
# get_keyphrases(category, number_documents = numberdocuments, number_phrases = numberphrases)

In [7]:
REQUEST = json.dumps({
    'body': {
        "category": "[2-GeneralCategorization/Health]http://opendata.aragon.es/def/ei2a/categorization#Health",
        "numberdocuments": 2,
        "numberphrases": 20
    }
})
print(REQUEST)

{"body": {"category": "[2-GeneralCategorization/Health]http://opendata.aragon.es/def/ei2a/categorization#Health", "numberdocuments": 2, "numberphrases": 20}}


In [8]:
# POST /keyphrasestextrank
req = json.loads(REQUEST)
category =  req['body']['category']
number_documents = req['body']['numberdocuments']
number_phrases = req['body']['numberphrases']

# print (req)
response = get_keyphrases(category, number_documents, number_phrases)

status=200
print (json.dumps({"list":response}))

{"list": ["informaci\u00f3n : 0.015860829922052386", "generada : 0.01255335664576223", "documentaci\u00f3n : 0.017825098609690407", "oficial : 0.011815382976537085", "garantizar : 0.01448252510594674", "universitario : 0.0175226998154406", "combatir : 0.012759401866496602", "realizar : 0.012245596333085907", "dimensi\u00f3n : 0.013565791474017258 internacional : 0.01735606756739859", "mediante : 0.012588591549890522", "intercambio : 0.015800238723435948", "choque : 0.012028959681046895", "circular : 0.01251599422218443", "impulsando : 0.014696932384263416", "tur\u00edstico : 0.013690800986032838", "acreditaci\u00f3n : 0.01641208321510232", "agencias : 0.012264302533991401", "superior : 0.012557185596314526", "compras : 0.011954950109142335", "inscrita : 0.012359219209195627"]}


In [9]:
# ResponseInfo POST /keyphrasestextrank
print(json.dumps({
    "status" : status,
    "headers" : {
        "Content-Type" : "application/json"
    }
}))

{"status": 200, "headers": {"Content-Type": "application/json"}}


In [10]:
#print nltk.data.path